# Baseline

In [11]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from pathlib import Path

from omegaconf import OmegaConf
from sklearn.model_selection import train_test_split

from cache import CACHE
from utils import (
    evaluate_score,
    load_data,
    save_predictions,
    setup_logging,
)

setup_logging()

PATH_DATA = Path("../data")
PATH_OUTPUT = Path("../output")

CACHE.init(cache_dir=PATH_OUTPUT / "cache")

[2025-05-21 13:54:09] INFO     cache:21    Cache directory: ../output/cache


## Load data

In [13]:
train_dataset = load_data(PATH_DATA / "training.csv")
test_dataset = load_data(PATH_DATA / "test.csv")

# Remove sentences longer than a threshold (e.g., 100 words)
max_sentence_length = 512
mask = train_dataset["sentence"].apply(lambda x: len(x.split()) <= max_sentence_length)
train_dataset = train_dataset[mask]

In [14]:
import nltk

with open(PATH_DATA / "stopwords.txt", "r") as f:
    stopwords = set(line.strip() for line in f if line.strip())

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/samuel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/samuel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_dataset["sentence"],
    train_dataset["label"],
    test_size=0.1,
    stratify=train_dataset["label"],
    random_state=0,
)

## Train pipeline

In [ ]:
config_bow_logreg = """
label_mapping: regression

vectorizer:
    type: CountVectorizer
    ngram_range: [1, 3]
    max_features: 10000

model:
    type: LogisticRegression
    C: 1.0
    max_iter: 100

preprocessing:
    - clean_whitespaces
    - internet
    - punctuation
    - contractions
    - remove_repeated_chars
    - remove_special_chars
"""

config_tfidf_logreg = """
label_mapping: regression

vectorizer:
    type: TfidfVectorizer
    ngram_range: [1, 3]
    max_features: 10000

model:
    type: LogisticRegression
    C: 1.0
    max_iter: 100

preprocessing:
    - clean_whitespaces
    - internet
    - punctuation
    - contractions
    - remove_repeated_chars
    - remove_special_chars
    - lemmatize
"""

active_config = OmegaConf.create(config_bow_logreg)

In [ ]:
from pipelines.classical_ml import ClassicalMLPipeline

pipeline = ClassicalMLPipeline(active_config)

train_predictions, val_predictions = pipeline.train(train_sentences, train_labels, val_sentences, val_labels)
score_train = evaluate_score(train_labels, train_predictions)
score_val = evaluate_score(val_labels, val_predictions)

print(f"Evaluation Score (training set): {score_train:.05f}")
print(f"Evaluation Score (validation set): {score_val:.05f}")

[2025-05-21 13:54:16] WARNING  py.warnings:110   /opt/homebrew/Caskroom/miniconda/base/envs/cil-project/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

Evaluation Score (training set): 0.83913
Evaluation Score (validation set): 0.80803


<!-- ## Make predictions on test data -->

In [18]:
test_ids = test_dataset.index
test_predictions = pipeline.predict(test_dataset["sentence"])

test_predictions

id
0         neutral
1         neutral
2         neutral
3        negative
4         neutral
           ...   
11946    positive
11947     neutral
11948    positive
11949    negative
11950    positive
Length: 11951, dtype: object

In [19]:
save_predictions(PATH_OUTPUT / "submissions" / "submission.csv", test_ids, test_predictions)

[2025-05-21 13:54:21] INFO     utils:103   Submission saved to '../output/submissions/submission.csv'.
